In [1]:
import numpy as np
import pandas as pd
import os,sys

In [2]:
# cutFlowFile = './HEPData-ins2080541-v1-csv/CutFlow,Chargino.csv'
cutFlowFile = './HEPData-ins2080541-v1-csv/CutFlow,Stau.csv'

In [3]:
# Get blocks for each parameter point (it is assumed they are separated by the '#: Coupling' tag)
with open(cutFlowFile,'r') as f:
    data = f.read()
    tag = '#: Model'
    blocks = data.split(tag)
    for ib,b in enumerate(blocks):
        blocks[ib] = tag.replace('# :','')+b
        break
# Skip header
blocks = blocks[1:]


In [4]:
dataDict = {}
for b in blocks:
    b = b.split('\n')
    mass = eval(b[0].split('=')[1].replace('TeV','*TeV').replace('GeV','*GeV'),
                {'TeV' : 1000.0,'GeV' : 1.0})
    tau = eval(b[1].split(',')[1].replace('ns','*ns'),
                {'ns' : 1})
    dataDict['mLLP'] = [mass]
    dataDict['tau_ns'] = [tau]
    dataType = b[2].split(',')[1]
    for l in b[3:]:
        l = l.strip()
        if not l: continue
        cut,val = l.rsplit(',',1)
        val = eval(val)
        if not cut in dataDict:
            dataDict[cut] = [[0.,0.]]
        if dataType == 'Events':
            dataDict[cut][0][0] = val
        else:
            dataDict[cut][0][1] = val

In [5]:
for k,v in dataDict.items():
    if isinstance(v[0],list):
        dataDict[k][0] = tuple(v[0])

In [6]:
# Create dataFrame
df = pd.DataFrame.from_dict(dataDict)
# Fix notation
df.rename(columns={'"$dE/dx\in[1.8,2.4] {MeV~cm^{2}/g}$"' : '(SR-Low - no mass Window)',
                   '$dE/dx>2.4 {MeV~cm^{2}/g}$' : '(SR-High - no mass Window)',
                  '"$m_{T}({track},{p}_{{T}}^{{ miss}}) > 130$ GeV"' : '$m_{T}({track},{p}_{{T}}^{{ miss}}) > 130$ GeV'},
                  inplace=True)

In [7]:
pickleFile = os.path.basename(cutFlowFile).replace(',','_').replace('.csv','.pcl')
df.to_pickle(pickleFile)

In [8]:
df

,mLLP,tau_ns,Total,Event and jet Cleaning,Trigger,Primary vertex,$E_{T}^{miss}>170$ GeV,Track in PV,$p_{T} > 50$ GeV,Track quality requirements,Track isolation,$p_{T} > 120$ GeV,Track momentum uncertainty,$|\eta|<1.8$,"$m_{T}({track},{p}_{{T}}^{{ miss}}) > 130$ GeV",Electron veto,Hadron/tau veto,(SR-Low - no mass Window),(SR-High - no mass Window)
0,400.0,10,"(258.0, 1.0)","(256.0, 0.991)","(129.0, 0.499)","(129.0, 0.499)","(87.0, 0.337)","(84.9, 0.328)","(83.7, 0.324)","(80.4, 0.311)","(79.4, 0.307)","(78.9, 0.305)","(78.3, 0.303)","(73.3, 0.284)","(60.6, 0.234)","(60.0, 0.232)","(58.8, 0.228)","(6.79, 0.026)","(6.18, 0.024)"
